# HW 7:
This homework we are going to start with a dataset ingest, some cleaning and some visualizations. Then move to Streamlit

## US Census Data

Load the data posted on the github repo "us-population-2010-2019.csv"

In [ ]:
# Your code here:
import pandas as pd
df = pd.read_csv("us-population-2010-2019-states-code.csv")
df.head()

Using a lambda function, add a column to the dataframe that provides standard two letter abbreviations to all of the US States. For example, Connecticut would be CT.
https://www.50states.com/abbreviations.htm
*hint* there is a python package called "us"

In [124]:
#Your code here
#!pip install us
import us

df["state_abbv"]= df["states"].apply(lambda x: us.states.lookup(x).abbr if us.states.lookup(x) != None else None)

df.head()

,states,states_code,id,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,state_abbv
0,Alabama,AL,1,"4,785,437","4,799,069","4,815,588","4,830,081","4,841,799","4,852,347","4,863,525","4,874,486","4,887,681","4,903,185",AL
1,Alaska,AK,2,"713,910","722,128","730,443","737,068","736,283","737,498","741,456","739,700","735,139","731,545",AK
2,Arizona,AZ,4,"6,407,172","6,472,643","6,554,978","6,632,764","6,730,413","6,829,676","6,941,072","7,044,008","7,158,024","7,278,717",AZ
3,Arkansas,AR,5,"2,921,964","2,940,667","2,952,164","2,959,400","2,967,392","2,978,048","2,989,918","3,001,345","3,009,733","3,017,804",AR
4,California,CA,6,"37,319,502","37,638,369","37,948,800","38,260,787","38,596,972","38,918,045","39,167,117","39,358,497","39,461,588","39,512,223",CA


Reshape your data, and create a new df called df_reshaped so that:

1. Convert 'year' column values to integers
2. Convert 'states' to strings
3. Get rid of the commas in the population numbers, and convert them to integers
4. Check your df_reshaped 

In [125]:
# Your code here:
#using melt instead of wide to long...
df_reshaped=pd.melt(df,id_vars=['id','states','states_code','state_abbv'],value_name='population',var_name='year')

#change year to int format
df_reshaped['year']=df_reshaped['year'].astype(int)

#replace "," with nothing and then format as integer
df_reshaped['population']=df_reshaped['population'].apply(lambda x: x.replace(",", ""))
df_reshaped['population']=df_reshaped['population'].astype(int)

#change states to string
df_reshaped['states']=df_reshaped['states'].astype(str)
df_reshaped.head()

,id,states,states_code,state_abbv,year,population
0,1,Alabama,AL,AL,2010,4785437
1,2,Alaska,AK,AK,2010,713910
2,4,Arizona,AZ,AZ,2010,6407172
3,5,Arkansas,AR,AR,2010,2921964
4,6,California,CA,CA,2010,37319502


Save your df_reshaped to a csv file

In [74]:
#Your code here:

#do not include index
df_reshaped.to_csv("df_reshaped.csv", index=False)

Subset your dataframe by selected_year = 2019 year create a new dataframe called df_selected_year


In [76]:
#Your code here:
df_selected_year = df_reshaped[df_reshaped['year'] == 2019]
df_selected_year.head()

,id,states,states_code,state_abbv,year,population
468,1,Alabama,AL,AL,2019,4903185
469,2,Alaska,AK,AK,2019,731545
470,4,Arizona,AZ,AZ,2019,7278717
471,5,Arkansas,AR,AR,2019,3017804
472,6,California,CA,CA,2019,39512223


Sort df_selected_year by population, from highest to lowest, and create a new df called "df_selected_year_sorted"

In [85]:
#Your code here:
df_selected_year_sorted = df_selected_year.sort_values(by ="population", ascending = False)
df_selected_year_sorted.head()

,id,states,states_code,state_abbv,year,population
472,6,California,CA,CA,2019,39512223
511,48,Texas,TX,TX,2019,28995881
477,12,Florida,FL,FL,2019,21477737
500,36,New York,NY,NY,2019,19453561
506,42,Pennsylvania,PA,PA,2019,12801989


In [89]:
# Here is a function to calculate population difference between selected and previous year
def calculate_population_difference(input_df, input_year):
    selected_year_data = input_df[input_df['year'] == input_year].reset_index()
    previous_year_data = input_df[input_df['year'] == input_year - 1].reset_index()
    selected_year_data['population_difference'] = selected_year_data.population.sub(previous_year_data.population, fill_value=0)
    selected_year_data['population_difference_absolute'] = abs(selected_year_data['population_difference'])
    return pd.concat([selected_year_data.states, selected_year_data.id, selected_year_data.population, selected_year_data.population_difference, selected_year_data.population_difference_absolute], axis=1).sort_values(by="population_difference", ascending=False)

#(Dili's edit: put in random year, show first 5 values)
df_population_difference_sorted = calculate_population_difference(df_reshaped, 2019)
df_population_difference_sorted.head()

,states,id,population,population_difference,population_difference_absolute
43,Texas,48,28995881,367215,367215
9,Florida,12,21477737,233420,233420
2,Arizona,4,7278717,120693,120693
33,North Carolina,37,10488084,106469,106469
10,Georgia,13,10617423,106292,106292


In [104]:
# Filter states with population difference > 50000
#Your code here
df_pop_diff = calculate_population_difference(df_reshaped, 2019)
df_greater_50000 = df_pop_diff[df_pop_diff.apply(lambda x: x['population_difference'] > 50000, axis = 1)]

In [107]:
# Calculate the % of States with population difference > 50000
#Your code here:

#number of states divided by the total number
len(df_greater_50000)/50

0.24

## Plots

### Heatmap: run the following code to see a heatmap

In [123]:
#!pip install altair
import altair as alt

alt.themes.enable("dark")

heatmap = alt.Chart(df_reshaped).mark_rect().encode(
        y=alt.Y('year:O', axis=alt.Axis(title="Year", titleFontSize=16, titlePadding=15, titleFontWeight=900, labelAngle=0)),
        x=alt.X('states:O', axis=alt.Axis(title="States", titleFontSize=16, titlePadding=15, titleFontWeight=900)),
        color=alt.Color('max(population):Q',
                         legend=alt.Legend(title=" "),
                         scale=alt.Scale(scheme="blueorange")),
        stroke=alt.value('black'),
        strokeWidth=alt.value(0.25),
        #tooltip=[
        #    alt.Tooltip('year:O', title='Year'),
        #    alt.Tooltip('population:Q', title='Population')
        #]
    ).properties(width=900
    #).configure_legend(orient='bottom', titleFontSize=16, labelFontSize=14, titlePadding=0
    #).configure_axisX(labelFontSize=14)
    ).configure_axis(
    labelFontSize=12,
    titleFontSize=12
    )

heatmap

alt.Chart(...)

### Choropleth: Run the following code to get a map of the population for the selected year above

In [ ]:
# Choropleth via Altair
#!pip install vega_datasets
import altair as alt
from vega_datasets import data

alt.themes.enable("dark")

states = alt.topo_feature(data.us_10m.url, 'states')

alt.Chart(states).mark_geoshape().encode(
    color=alt.Color('population:Q', scale=alt.Scale(scheme='blues')),   # scale=color_scale
    stroke=alt.value('#154360')
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df_selected_year, 'id', list(df_selected_year.columns))
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
)



Pycharm:
Create a pdf reader streamlit app in pycharm using the following code.Upload some pdfs and perform some queries. 
Below capture 5 queries and 5 responses. Check to make sure the responses are accurate. See if you can get your app to fail. Analyze why.

In [1]:
#!pip install streamlit pypdf2 langchain python-dotenv faiss-cpu openai huggingface_hub langchain
import streamlit as st
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain



def get_pdf_text(pdf_docs):

    text: str = ''
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks= text_splitter.split_text(text)
    return chunks

def get_vectorstore(text_chunks):
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore

def get_conversation_chain(vectorstore):
    llm = ChatOpenAI()
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain

def handle_userinput(user_question):
    response = st.session_state.conversation({'question':user_question})
    st.session_state.chat_history = response['chat_history']
    for i, message in enumerate(st.session_state.chat_history):
        if i%2 ==0:
            st.write(f'Human Question: {message.content}')
        else:
            st.write(f'AI Response: {message.content}')

        
def main():
    load_dotenv()
    st.set_page_config(page_title="Chat with PDFs", page_icon=":scalpel:")

    if "conversation" not in st.session_state:
        st.session_state.conversation = None
    if "chat_history" not in st.session_state:
        st.session_state.chat_history= None

    st.header("Chat with PDFs :medical_symbol:")
    user_question = st.text_input("Ask a question about your documents:")
    if user_question:
        handle_userinput(user_question)

    with st.sidebar:
        st.subheader("Your documents")
        pdf_docs = st.file_uploader("Upload your PDFs here and click on 'Process'", accept_multiple_files=True)
        if st.button('Process'):
            with st.spinner("Processing..."):

            #get pdf texts
                raw_text = get_pdf_text(pdf_docs)
                #st.write(raw_text)

            #chunk pdfs
                text_chunks = get_text_chunks(raw_text)
                #st.write(text_chunks)

            #create vector store with embeddings
                vectorstore = get_vectorstore(text_chunks)

            # create conversation chain
                st.session_state.conversation = get_conversation_chain(vectorstore)


if __name__=='__main__':
    main()


2024-04-26 22:52:00.621 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2024-04-26 22:52:00.821 
  command:

    streamlit run /Users/dili/anaconda3/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


#### 1. Paper given: Research paper on marijuana legalization

**Query**: What was the main aim of the paper. List the methology, question, data used and how the data was constructed, as well as the findings and potential next steps

**AI Response**: The main aim of the paper was to estimate the effects of both recreational and medical marijuana legalization on hard drug use and DUIs, specifically in counties bordering states with marijuana laws.

Methodology:

The study used a difference-in-differences design.
Implemented a robust identification strategy to control for county-linear time trends.
Standard errors were clustered at the county level to address concerns related to county-level confounding.
Question:

The study aimed to understand the impact of marijuana laws on possession of hard drugs, possession of marijuana, and DUI cases in counties bordering states with marijuana laws.
Data Used and Data Construction:

Data sources included the Uniform Crime Reports and the US Census Bureau.
The data was constructed to focus on arrests by county.
The study used county-by-year fixed effects to control for unobserved time-varying effects at the county level.
Findings:

Counties bordering states with any marijuana law experienced an increase in hard drug arrests following the enactment of marijuana laws.
No conclusive evidence was found for an effect on possession of marijuana or DUI cases in the study.
Potential Next Steps:

The study recommended that future research should include additional controls to account for potential biases, such as the level of police activity in counties bordering legalized states.
Exploring the impact of marijuana laws on other types of crimes or behaviors could be a potential next step.

`Remark`: They are correct

#### 2. Paper given: My resume

**Query**: Where did this person work as of 2024

**AI Response**: As of 2024, this person worked as a Graduate Research Assistant at Yale University under Dr. Jonas Lieber.

`Remark`: They are correct

#### 3. Paper given: Lecture note on expenditure function

**Query**: Give a short overview of the expenditure function, include equations and additional information I may need to know to understand
Human Question: Give a short overview of the expenditure function, include equations and additional information I may need to know to understand

**AI Response**: The expenditure function is a concept in economics that tells us the minimum amount of money (expenditure) needed to achieve a given level of utility. It is derived from the consumer's problem of minimizing total expenditure on goods while attaining a specific utility level. The expenditure function is denoted as E(px, py, Uo), where px and py are the prices of goods x and y, and Uo is the utility level.

Formally, the expenditure function is derived by solving the dual consumer's problem using a Lagrangian with the constraint U(x, y) = Uo. This is a minimization problem where the objective is to minimize pxx + pyy subject to the utility constraint. The Lagrange multiplier helps find the correct expenditure amount.

The expenditure function is important in welfare economics as it helps determine the compensation needed to restore an original level of utility. It is homogenous of degree one in prices (px, py) and can be used to calculate the minimum expenditure required for different utility levels. The Envelope Theorem is also used to understand the relationship between the expenditure function and Hicksian demands.

`Remark`: Pretty good, but I had to run it 3 times! The first 2 times, the response was "I don't know"

#### 4. Paper given: Syllabus on my machine learning in economics class

**Querty**: What are the things I am going to learn in my machine learning in economics class

**AI Response**: Dili Maduabum's coursework in Machine Learning likely covers topics such as regression analysis, classification algorithms, clustering techniques, neural networks, decision trees, and more. However, the specific topics covered in their Machine Learning in Economics class at Yale University are not explicitly mentioned in the provided context.

`Remark`: I would have preferred more detail but I wasn't detailed either so it did okay.

#### 5. Paper given: "Social Networks and the Dynamics of Labour Market Outcomes: Evidence from Refugees Resettled in the U.S. " by Lori Beaman 

**Querty**: Explain the mathematical model used in Social Networks and the Dynamics of Labour Market Outcome. Derive the steps

**AI Response**: I don't have access to the specific mathematical model used in the study "Social Networks and the Dynamics of Labour Market Outcomes: Evidence from Refugees Resettled in the U.S." by Lori Beaman. The study discusses a theoretical model of job information transmission that shows the relationship between social network size and labour market outcomes, emphasizing that the impact can vary for different cohorts within the network. The model suggests that an increase in network size may have negative effects on some groups while benefiting others. To test this prediction, the study utilized data on political refugees resettled in the U.S.

The research focused on how social networks influence access to local labor markets for refugees, emphasizing the importance of the size and structure of the network, particularly the length of tenure of network members in the U.S. The study found that while network architecture can improve labour market outcomes through information transmission, there is also a competitive effect within the network that negatively impacts outcomes. The study highlights the dynamic relationship between employment, wages, and social network structure, suggesting that understanding the tenure composition and other structural dimensions of social networks is crucial in assessing their full effect on labor market outcomes.

`Remark`: It is spot on for the finding, but was not good with the math. This will be very helpful for literature reviews